# Light cross sectional flux (LCSF) method
This notebook demonstrates how to quantify CO$_2$ and NO$_x$ emissions from point sources using synthetic CO2M observations for a power plant and for a city.
The method used to quantify emissions is derived from a cross-sectional flux estimation method illustrated in Zheng et al., 2020 (https://doi.org/10.5194/acp-20-8501-2020) and Chevallier et al., 2020 (https://doi.org/10.1029/2020GL090244)

In [ ]:
import os

import cartopy.crs as ccrs
import numpy as np
import pandas as pd
import ucat
import xarray as xr


import warnings
warnings.filterwarnings('ignore')

# import and setup matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 100

import ddeq

Define coordinate reference systems used by the tool, the SMARTCARB model domain and the path to SMARTCARB data:

In [ ]:
from ddeq.smartcarb import DOMAIN

# Path to test dataset incl in ddeq
from ddeq import DATA_PATH

Read list of point sources in the SMARTCARB model domain from "sources.csv" file.

In [ ]:
# list of point sources
sources = ddeq.misc.read_point_sources()
sources = sources.sel(source=['Janschwalde', 'Boxberg'])
sources

## Synthetic satellite observations

Synthetic satellite observations are available from the SMARTCARB project (https://doi.org/10.5281/zenodo.4048227). The `ddeq` package can read the data files and automatically applies random noise and cloud filters to the observations. The code also fixes some issues with the dataset such as wrong emissions for industry in Berlin in January and July. It is also possible to scale the anthropogenic model tracers: 

In [ ]:
filename = os.path.join(DATA_PATH, 'Sentinel_7_CO2_2015042311_o1670_l0483.nc')
data = ddeq.smartcarb.read_level2(filename, co2_noise_scenario='low', no2_noise_scenario='high')

## Light cross-sectional flux estimation

### Parameters of the method
The operations of the light cross-sectional method for estimating emissions are defined by several parameters.
Complete list can be found in the function ddeq.lcs.estimate_emissions. A shorter list can be provided by the user as the example below

In [ ]:
lcs_params={}

# if float: Altitude at or below (if wind_alti_avg set to True) which effective
# winds for emission computation are extracted.
# if set to 'GNFR-A': ERA-5 averaged winds depending on profile emissions are used as effective winds
lcs_params['alti_ref'] = 'GNFR-A' # Or 'GNFR-A'

# if effective winds for emissions computation are averaged between the surface and the reference altitude
lcs_params['wind_alti_avg'] = False

# Wind product. So far the implementation has been done for SMARTCARB and ERA-5 winds
lcs_params['wind_product'] = 'SMARTCARB' # ERA5 || SMARTCARB

# Tracer gases used for the emission estimations
# If NO2 l2 data are used, NOx emissions are estimated.
# If CO2_with_NO2 is prescribed, NOx and CO2 emissions from CO2 data only are also provided
lcs_params['tracer_gases'] = ['CO2']  # ['CO2','NO2','CO2_with_NO2']

lcs_params['use_prior'] = False

### Extracting wind fields
Wind files should correspond to the L2 satellite data file and to the chosen wind product.

In [ ]:
if lcs_params['wind_product'] == 'ERA5':
    pass
else:
    wind_filename = os.path.join(DATA_PATH, 'SMARTCARB_winds_2015042311.nc')

winds = ddeq.wind.read_field(wind_filename, altitude=lcs_params['alti_ref'], average_below=False, product=lcs_params['wind_product'])

Emissions of the sources are estimated by the function ddeq.lcs.estimate_emissions. 
By default, the function returns for all sources the emission estimates, their precision, and the distances
to the source of the points where emissions are estimated. If all_diags set to True, additional information relative
to the line densities are also provided for the graphical representation of the results.

In [ ]:
lcs_params = {}
lcs_params['verbose'] = False
lcs_params['f_NOx_NO2'] = 1.32

lcs_params['n_min_fit_pts'] = 10
#lcs_params['fit_pt_slice_width'] = 15


# Q prior with 0.1-1.9 bounds
lcs_params['use_prior'] = True
priors = dict(
    (name, {
        'NO2': {'Q': 1.0, 'tau': 3600*4.0},
        'CO2': {'Q': 1000.0, 'tau': np.nan}})
    for name in sources.source.values
)

In [ ]:
res_no2 = ddeq.lcsf.estimate_emissions(data, winds, sources, priors=priors,
                                       lcs_params=lcs_params, gases=['NO2'],
                                       fit_backgrounds=[False], all_diags=True)

res_co2 = ddeq.lcsf.estimate_emissions(data, winds, sources, priors=priors,
                                       lcs_params=lcs_params, gases=['CO2'],
                                       fit_backgrounds=[True], all_diags=True)

res_both = ddeq.lcsf.estimate_emissions(data, winds, sources,
                                  lcs_params=lcs_params, gases=['NO2', 'CO2'], priors=priors,
                                  fit_backgrounds=[False, True], all_diags=True)

Printing of the results (Only works for SMARTCARB dataset)

In [ ]:
#               CO2 (Mt/yr)  CO2 true (Mt/yr)  NO2 (kt/yr)   NO2 true (kt/yr)
# Janschwalde   45.2         42.4              46.9          34.2
# Boxberg                    24.2              21.0          19.6
#
ddeq.lcsf.make_results_table_for_smartcarb(res_both, sources, ['CO2', 'NO2'], data)

### Plotting results for a given source

In [ ]:
#Chosen Source
source_name = 'Janschwalde'
emi_time = pd.Timestamp(data.time.values)

true_emis = {}
for gas in ['CO2', 'NO2']:
    true_emis[gas] = ddeq.smartcarb.read_true_emissions(gas, source_name, time=emi_time)
    true_emis[gas] = ucat.convert_mass_per_time_unit(
        true_emis[gas],
        'kg/s',
        'Mt/a' if gas == 'CO2' else 'kt/a'
    )

fig = ddeq.vis.plot_lcsf_result(source_name, res_both, data, sources,
                                gases=['CO2', 'NO2'], true_emis=true_emis)
